In [1]:
import os
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [2]:
load_dotenv()

True

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [4]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [5]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-3-small"
)

In [6]:
chroma_client = chromadb.HttpClient(host='localhost', port=8000)

In [7]:
collection = chroma_client.get_or_create_collection('blog', embedding_function=openai_ef)

In [8]:
vectorstore = Chroma(client=chroma_client, collection_name='blog', embedding_function=OpenAIEmbeddings())

In [9]:
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold":0.5})

In [10]:
prompt = hub.pull("rlm/rag-prompt")

In [11]:
example_message = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

In [12]:
print(example_message[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [14]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
for chunk in rag_chain.stream("What is Rater Agreement?"):
    print(chunk, end="", flush=True)

d:\Programacion\langchain\venv\Lib\site-packages\langchain_core\vectorstores\base.py:784: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'id': 46, 'metadata': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'page_content': 'Here are a sample conversation for task clarification sent to OpenAI ChatCompletion endpoint used by GPT-Engineer. The user inputs are wrapped in {{user input text}}.\n[\n  {\n    "role": "system",\n    "content": "You will read instructions and not carry them out, only seek to clarify them.\\nSpecifically you will first summarise a list of super short bullets of areas that need clarification.\\nThen you will pick one clarifying question, and wait for an answer from the user.\\n"\n  },\n  {\n    "role": "user",\n    "content": "We are writing {{a Super Mario game in python. MVC components split in separate files. Keyboard control.}}\\n"\n  },\n  {\n    "role": "assistant",', 'tokens': 'here are sample conversation for task cl

Rater agreement is a statistical measure that assesses the consistency of judgments made by two or more raters. It quantifies the level of agreement between raters on a particular task, such as coding text or evaluating performance.  Higher rater agreement indicates greater consistency and reliability in the ratings. 


#### Crear una Chain personalizada

In [15]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [16]:
system_prompt = (
    'Usted es un asistente para tareas de respuesta a preguntas. '
    'Si el texto de entrada esta en un idioma diferente al español, traduzca el texto al español.'
    'Utilice las siguientes piezas de contexto recuperado para responder '
    'la pregunta. Si no sabes la respuesta, di que '
    'No lo sé. Utilice tres oraciones como máximo y mantenga el '
    'respuesta concisa.'
    '\n\n'
    '{context}'
)

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [18]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain_2 = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain_2.invoke({"input": "What is Task Decomposition? get me the examples of task decomposition"})
print(response["answer"])

d:\Programacion\langchain\venv\Lib\site-packages\langchain_core\vectorstores\base.py:784: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'id': 18, 'metadata': 'https://lilianweng.github.io/posts/2024-02-05-human-data-quality/', 'page_content': 'Fig. 5. A taxonomy of causes for rater disagreement. (Image source: Zhang et al. 2023)\nDisagreement deconvolution relies on probabilistic graph modeling:\n\nEstimate how often an annotator returns non-primary labels, $p_\\text{flip}$\nPer sample, get an adjusted label distribution $p^*$ of primary labels based on $p_\\text{flip}$\nSample from $p^*$ as a new test set.\nMeasure performance metrics against the new test set.\n\nGiven $C$-category classification, the sampling process of the generative model is stated as follows:\n\n$$\n\\begin{aligned}\ny^*\\mid x &\\sim \\text{Categorial}([C], p^*(y\\mid x)) \\\\\ny_\\text{other}\\mid y^* &\\sim \\text{Categorial}([C]\\setminus\\{y^*\\}, \\frac{1}{C-1}) \\\\\nz_\\te

Task decomposition is the process of breaking down a large, complex task into smaller, more manageable subtasks. This makes the task easier to understand, plan, and execute. 

For example, writing a book can be decomposed into tasks like outlining, researching, drafting, editing, and proofreading. Building a house can be decomposed into tasks like laying the foundation, framing the walls, installing plumbing, and painting the interior. 

